#### This notebook prepare the Bank Dataset and do the 5 fold split

After we get the 5 fold data, we then save it to the csv file

In [1]:
%matplotlib inline
# Load all necessary packages
import sys
sys.path.append("../")
import numpy as np
from tqdm import tqdm
from warnings import warn

from aif360.datasets import BinaryLabelDataset
from aif360.datasets import BankDataset
from aif360.metrics import ClassificationMetric, BinaryLabelDatasetMetric
from aif360.metrics.utils import compute_boolean_conditioning_vector
from aif360.algorithms.postprocessing.reject_option_classification\
        import RejectOptionClassification
from common_utils import compute_metrics

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

from IPython.display import Markdown, display
import matplotlib.pyplot as plt
from ipywidgets import interactive, FloatSlider
from sklearn.linear_model import Lasso
import pickle
import pandas as pd

`load_boston` has been removed from scikit-learn since version 1.2.

The Boston housing prices dataset has an ethical problem: as
investigated in [1], the authors of this dataset engineered a
non-invertible variable "B" assuming that racial self-segregation had a
positive impact on house prices [2]. Furthermore the goal of the
research that led to the creation of this dataset was to study the
impact of air quality but it did not give adequate demonstration of the
validity of this assumption.

The scikit-learn maintainers therefore strongly discourage the use of
this dataset unless the purpose of the code is to study and educate
about ethical issues in data science and machine learning.

In this special case, you can fetch the dataset from the original
source::

    import pandas as pd
    import numpy as np

    data_url = "http://lib.stat.cmu.edu/datasets/boston"
    raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
    data = np.hstack([raw_df.values[::2, :], raw_df

#### Load dataset and specify options

for the bank dataset, age is the protected attribute.

  By default, this code converts the 'age' attribute to a binary value
        where privileged is `25 <= age < 60` and unprivileged is `age < 25` or `age >= 60`
        as suggested in Le Quy, Tai, et al. [1].

        References:
            .. [1] Le Quy, Tai, et al. "A survey on datasets for fairness‐aware machine 
            learning." Wiley Interdisciplinary Reviews: Data Mining and Knowledge 
            Discovery 12.3 (2022): e1452.

However, in Function Composition in Trustworthy Machine Learning:
Implementation Choices, Insights, and Questions 2023 feb, they  use ‘age’ (‘under 25’ is privileged) as a sensitive attribute.

In [2]:
dataset_orig = BankDataset()
privileged_groups = [{'age': 1}]
unprivileged_groups = [{'age': 0}]
# Metric used (should be one of allowed_metrics)
metric_name = "Equal opportunity difference"
#random seed for calibrated equal odds prediction
random_seed = 12345679
np.random.seed(random_seed)
# Upper and lower bound on the fairness metric used
metric_ub = 0.05
metric_lb = -0.05

In [3]:
# 5 fold cross validation
Z =  dataset_orig.split(5, shuffle=True,seed = random_seed)
# i th fold
dataset_train1 = Z[0].copy()
dataset_train1.features = np.concatenate((Z[0].features,Z[1].features,Z[2].features,Z[3].features),axis=0)
dataset_train1.scores = np.concatenate((Z[0].scores,Z[1].scores,Z[2].scores,Z[3].scores),axis=0)
dataset_train1.labels = np.concatenate((Z[0].labels,Z[1].labels,Z[2].labels,Z[3].labels),axis=0)
dataset_train1.protected_attributes = np.concatenate((Z[0].protected_attributes,Z[1].protected_attributes,Z[2].protected_attributes,Z[3].protected_attributes),axis=0)
dataset_train1.instance_weights = np.concatenate((Z[0].instance_weights,Z[1].instance_weights,Z[2].instance_weights,Z[3].instance_weights),axis=0)
dataset_train1.instance_names = np.concatenate((Z[0].instance_names,Z[1].instance_names,Z[2].instance_names,Z[3].instance_names),axis=0)
dataset_train1.metadata = Z[0].metadata.copy()
dataset_test1= Z[4].copy()

dataset_train2 = Z[1].copy()
dataset_train2.features = np.concatenate((Z[1].features,Z[2].features,Z[3].features,Z[4].features),axis=0)
dataset_train2.scores = np.concatenate((Z[1].scores,Z[2].scores,Z[3].scores,Z[4].scores),axis=0)
dataset_train2.labels = np.concatenate((Z[1].labels,Z[2].labels,Z[3].labels,Z[4].labels),axis=0)
dataset_train2.protected_attributes = np.concatenate((Z[1].protected_attributes,Z[2].protected_attributes,Z[3].protected_attributes,Z[4].protected_attributes),axis=0)
dataset_train2.instance_weights = np.concatenate((Z[1].instance_weights,Z[2].instance_weights,Z[3].instance_weights,Z[4].instance_weights),axis=0)
dataset_train2.instance_names = np.concatenate((Z[1].instance_names,Z[2].instance_names,Z[3].instance_names,Z[4].instance_names),axis=0)
dataset_train2.metadata = Z[1].metadata.copy()
dataset_test2= Z[0].copy()

dataset_train3 = Z[2].copy()
dataset_train3.features = np.concatenate((Z[2].features,Z[3].features,Z[4].features,Z[0].features),axis=0)
dataset_train3.scores = np.concatenate((Z[2].scores,Z[3].scores,Z[4].scores,Z[0].scores),axis=0)
dataset_train3.labels = np.concatenate((Z[2].labels,Z[3].labels,Z[4].labels,Z[0].labels),axis=0)
dataset_train3.protected_attributes = np.concatenate((Z[2].protected_attributes,Z[3].protected_attributes,Z[4].protected_attributes,Z[0].protected_attributes),axis=0)
dataset_train3.instance_weights = np.concatenate((Z[2].instance_weights,Z[3].instance_weights,Z[4].instance_weights,Z[0].instance_weights),axis=0)
dataset_train3.instance_names = np.concatenate((Z[2].instance_names,Z[3].instance_names,Z[4].instance_names,Z[0].instance_names),axis=0)
dataset_train3.metadata = Z[2].metadata.copy()
dataset_test3= Z[1].copy()

dataset_train4 = Z[3].copy()
dataset_train4.features = np.concatenate((Z[3].features,Z[4].features,Z[0].features,Z[1].features),axis=0)
dataset_train4.scores = np.concatenate((Z[3].scores,Z[4].scores,Z[0].scores,Z[1].scores),axis=0)
dataset_train4.labels = np.concatenate((Z[3].labels,Z[4].labels,Z[0].labels,Z[1].labels),axis=0)
dataset_train4.protected_attributes = np.concatenate((Z[3].protected_attributes,Z[4].protected_attributes,Z[0].protected_attributes,Z[1].protected_attributes),axis=0)
dataset_train4.instance_weights = np.concatenate((Z[3].instance_weights,Z[4].instance_weights,Z[0].instance_weights,Z[1].instance_weights),axis=0)
dataset_train4.instance_names = np.concatenate((Z[3].instance_names,Z[4].instance_names,Z[0].instance_names,Z[1].instance_names),axis=0)
dataset_train4.metadata = Z[3].metadata.copy()
dataset_test4= Z[2].copy()

dataset_train5 = Z[4].copy()
dataset_train5.features = np.concatenate((Z[4].features,Z[0].features,Z[1].features,Z[2].features),axis=0)
dataset_train5.scores = np.concatenate((Z[4].scores,Z[0].scores,Z[1].scores,Z[2].scores),axis=0)
dataset_train5.labels = np.concatenate((Z[4].labels,Z[0].labels,Z[1].labels,Z[2].labels),axis=0)
dataset_train5.protected_attributes = np.concatenate((Z[4].protected_attributes,Z[0].protected_attributes,Z[1].protected_attributes,Z[2].protected_attributes),axis=0)
dataset_train5.instance_weights = np.concatenate((Z[4].instance_weights,Z[0].instance_weights,Z[1].instance_weights,Z[2].instance_weights),axis=0)
dataset_train5.instance_names = np.concatenate((Z[4].instance_names,Z[0].instance_names,Z[1].instance_names,Z[2].instance_names),axis=0)
dataset_train5.metadata = Z[4].metadata.copy()
dataset_test5= Z[3].copy()

#### Save the split dataset

In [6]:
protected_name = "age"
dataset_train1.convert_to_dataframe()[0].to_csv("Huangrui/bank/{}/bank_train1.csv".format(protected_name),index=False)
dataset_test1.convert_to_dataframe()[0].to_csv("Huangrui/bank/{}/bank_test1.csv".format(protected_name),index=False)
dataset_train2.convert_to_dataframe()[0].to_csv("Huangrui/bank/{}/bank_train2.csv".format(protected_name),index=False)
dataset_test2.convert_to_dataframe()[0].to_csv("Huangrui/bank/{}/bank_test2.csv".format(protected_name),index=False)
dataset_train3.convert_to_dataframe()[0].to_csv("Huangrui/bank/{}/bank_train3.csv".format(protected_name),index=False)
dataset_test3.convert_to_dataframe()[0].to_csv("Huangrui/bank/{}/bank_test3.csv".format(protected_name),index=False)
dataset_train4.convert_to_dataframe()[0].to_csv("Huangrui/bank/{}/bank_train4.csv".format(protected_name),index=False)
dataset_test4.convert_to_dataframe()[0].to_csv("Huangrui/bank/{}/bank_test4.csv".format(protected_name),index=False)
dataset_train5.convert_to_dataframe()[0].to_csv("Huangrui/bank/{}/bank_train5.csv".format(protected_name),index=False)
dataset_test5.convert_to_dataframe()[0].to_csv("Huangrui/bank/{}/bank_test5.csv".format(protected_name),index=False)